# Part 1

## Installing Spark

In [3]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import findspark
from pyspark.sql import SparkSession

# Initialize findspark
findspark.init()

# Create SparkSession
spark = SparkSession.builder \
                    .appName("Example") \
                    .getOrCreate()

spark

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Hit:3 https://dl.yarnpkg.com/debian stable InRelease                           
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                      
Hit:5 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Hit:8 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Fetched 114 kB in 1s (106 kB/s)                 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
42 packages can be upgraded. Run 'apt list --upgradable' to see them.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
tar: sp

24/02/07 13:38:37 WARN Utils: Your hostname, codespaces-4f3c52 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/02/07 13:38:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/07 13:38:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## DF

In [1]:
import requests

# List of URLs
urls = {
    "Arrest": "https://data.cityofnewyork.us/resource/uip8-fykc.json",
    "Poverty": "https://data.cityofnewyork.us/resource/cts7-vksw.json",
    "Collisions": "https://data.cityofnewyork.us/resource/bm4k-52h4.json",
    "Education": "https://data.cityofnewyork.us/resource/2dzy-e7cu.json"
}

# Function to extract data from each URL
def extract_data(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Data downloaded successfully from {url} and saved as {filename}")
    else:
        print(f"Failed to download data from {url}. Status code: {response.status_code}")

# Iterate over each URL and extract data
for name, url in urls.items():
    filename = f"/workspaces/Procesamiento-Datos/Project/Data/{name.lower()}.json"
    extract_data(url, filename)



Data downloaded successfully from https://data.cityofnewyork.us/resource/uip8-fykc.json and saved as /workspaces/Procesamiento-Datos/Project/Data/arrest.json
Data downloaded successfully from https://data.cityofnewyork.us/resource/cts7-vksw.json and saved as /workspaces/Procesamiento-Datos/Project/Data/poverty.json
Data downloaded successfully from https://data.cityofnewyork.us/resource/bm4k-52h4.json and saved as /workspaces/Procesamiento-Datos/Project/Data/collisions.json
Data downloaded successfully from https://data.cityofnewyork.us/resource/2dzy-e7cu.json and saved as /workspaces/Procesamiento-Datos/Project/Data/education.json


In [4]:
# Read JSON data into a DataFrame
Arrest_df= spark.read.json("/workspaces/Procesamiento-Datos/Project/Data/arrest.json" , multiLine=True)
Collisions_df= spark.read.json("/workspaces/Procesamiento-Datos/Project/Data/collisions.json" , multiLine=True)
Education_df= spark.read.json("/workspaces/Procesamiento-Datos/Project/Data/education.json" , multiLine=True)
Poverty_df= spark.read.json("/workspaces/Procesamiento-Datos/Project/Data/poverty.json" , multiLine=True)

In [5]:
# Show the DataFrame
Poverty_df.show()

24/02/07 13:38:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+----+----+---+-------------+---+----------+----+----+-------------+-------------+--------+-----------+--------+-----------+-------------+---------+-------------+----------+---------------+---------+----------+--------+---+---------+----+----+---+---------+----+---+-------------+---------------+----------+----------------+------------+-------------+------+------+----------+-----------+---------------+-----+---+---------+---------+---+----+----+---------+--------+---+-------+--------+---------+---+---------------+---------+----+----+----+
|agecateg|agep|boro|cit|citizenstatus|dis|educattain| eng| esr|est_childcare|est_commuting|est_eitc|est_ficatax|est_heap|est_housing|est_incometax| est_moop|est_nutrition|est_povgap|est_povgapindex|ethnicity|famtype_pu|ftptwork|hht| intp_adj|jwtr|lanx|mar| mrgp_adj| msp| np|nycgov_income|nycgov_pov_stat|nycgov_rel|nycgov_threshold|off_pov_stat|off_threshold|oi_adj|pa_adj|povunit_id|povunit_rel|pretaxincome_pu|pwgtp|rel| retp_adj| rntp_adj|sch|

24/02/07 13:38:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Bonus

## Web Scrapping

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website
url = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoodpop.htm"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table with the specified class
table = soup.find('table', {'class': 'light_table right'})

# Extract the table rows
rows = table.find_all('tr')

# Extracting data from each row
data = []
for row in rows:
    cols = row.find_all(['th', 'td'])
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Create a DataFrame from the extracted data
columns = data[0]  # Assuming the first row contains column headers
df = pd.DataFrame(data[1:], columns=columns)

# Display the head of the DataFrame
print(df.head())


  Borough                                     region   Males Females  \
0   Bronx         Riverdale, Fieldston & Kingsbridge  52,133  61,937   
1               Wakefield, Williamsbridge & Woodlawn  65,087  77,848   
2             Co-op City, Pelham Bay & Schuylerville  55,615  65,929   
3              Pelham Parkway, Morris Park & Laconia  61,233  67,896   
4          Belmont, Crotona Park East & East Tremont  75,963  87,740   

  Total Population  
0          114,070  
1          142,935  
2          121,544  
3          129,130  
4          163,704  


## API

In [6]:
%pip install pyowm


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps


In [18]:
import pandas as pd

In [ ]:
from pyowm.owm import OWM
owm = OWM('db5829c814693c6429a0ea9db06f3dce')

mgr = owm.weather_manager()


# Function to retrieve weather data for a city
def get_weather(city):
    observation = mgr.weather_at_place(city)
    w = observation.weather
    return {
        'City': city,
        'Detailed Status': w.detailed_status,
        'Wind Speed': w.wind()['speed'],
        'Wind Direction': w.wind()['deg'],
        'Humidity': w.humidity,
        'Temperature': w.temperature('celsius')['temp'],
        'Max Temperature': w.temperature('celsius')['temp_max'],
        'Min Temperature': w.temperature('celsius')['temp_min'],
        'Rain': w.rain,
        'Heat Index': w.heat_index,
        'Clouds': w.clouds
    }

# List of important cities in the United States
important_cities_us = [
    'New York,US', 'Los Angeles,US', 'Chicago,US', 'Houston,US', 'Phoenix,US',
    'Philadelphia,US', 'San Antonio,US', 'San Diego,US', 'Dallas,US', 'San Jose,US',
    'Austin,US', 'Jacksonville,US', 'San Francisco,US', 'Columbus,US', 'Fort Worth,US',
    'Indianapolis,US', 'Charlotte,US', 'Seattle,US', 'Denver,US', 'Washington,US',
    'Boston,US', 'El Paso,US', 'Nashville,US', 'Detroit,US', 'Oklahoma City,US',
    'Portland,US', 'Las Vegas,US', 'Memphis,US', 'Louisville,US', 'Milwaukee,US'
]

# Retrieve weather data for each city
weather_data = [get_weather(city) for city in important_cities_us]

# Create DataFrame from the weather data
df_weather = pd.DataFrame(weather_data)


In [ ]:
# Display the DataFrame
df_weather.head()